In [14]:
import pandas as pd

pd.read_csv('../cornell/Cornell_2019_sampled.csv')

,Unnamed: 0,body,emo_pred_pos,emo_pred_neu,emo_pred_neg
0,0,Walk,0.1892,0.6907,0.1201
1,1,"Haha, didn't you sell me your switch two years...",0.0768,0.8144,0.1088
2,2,"sweet, that sounds like a plan",0.6314,0.3402,0.0284
3,3,They do warn you when they do fire inspections...,0.0272,0.6129,0.3600
4,4,hmmm i see thanks. I would buy a fire proof on...,0.0887,0.4546,0.4568
...,...,...,...,...,...
20558,20558,Wtf,0.0492,0.3474,0.6033
20559,20559,What a coincidence! I am in Ithaca visiting f...,0.4982,0.4794,0.0223
20560,20560,[deleted],0.0725,0.6524,0.2751
20561,20561,"In my 4 years at Cornell, I don't think classe...",0.4485,0.5080,0.0435
